# Build your own quantum computer simulator

## Section 1: Quantum States

A single qubit is represented by a normalized complex vector

$$
|\psi\rangle =
\begin{pmatrix}
\alpha \
\beta
\end{pmatrix},
\qquad
|\alpha|^2 + |\beta|^2 = 1.
$$

The computational basis states are

$$
|0\rangle =
\begin{pmatrix}
1 \ 0
\end{pmatrix},
\qquad
|1\rangle =
\begin{pmatrix}
0 \ 1
\end{pmatrix}.
$$

A general superposition state is written as

$$
|\psi\rangle = \alpha |0\rangle + \beta |1\rangle =
\begin{pmatrix}
\alpha \ \beta
\end{pmatrix}.
$$

Normalization ensures

$$
|\psi\rangle ; \mapsto ;
\frac{1}{\sqrt{|\alpha|^2 + |\beta|^2}}
\begin{pmatrix}
\alpha \ \beta
\end{pmatrix}.
$$

For multiple qubits, we use tensor products (Kronecker products in code):

$$
|\psi_{1\ldots n}\rangle = |\psi_1\rangle \otimes |\psi_2\rangle \otimes \cdots \otimes |\psi_n\rangle.
$$

In [ ]:
import numpy as np

# 0 and 1 qubit states
def ket0():
    return np.array([1, 0], dtype=complex)

def ket1():
    return np.array([0, 1], dtype=complex)

# Normalize a state vector
def normalize(psi: np.ndarray) -> np.ndarray:
    norm = np.linalg.norm(psi)
    if norm == 0:
        raise ValueError("Cannot normalize the zero vector.")
    return psi / norm

# General superposition state
def superposition(alpha: complex, beta: complex) -> np.ndarray:
    psi = normalize(np.array([alpha, beta], dtype=complex))
    return psi

# Tensor product of multiple operators
def kron_all(*arrays) -> np.ndarray:
    out = np.array([1], dtype=complex)
    for a in arrays:
        out = np.kron(out, a)
    return out


## Section 2: One-Qubit Gates

All single-qubit gates are $2\times 2$ unitary matrices acting on state vectors by matrix multiplication.

Identity and Pauli Matrices

$$
I =
\begin{pmatrix}
[1 & 0] \
[0 & 1]
\end{pmatrix}, \quad
X =
\begin{pmatrix}
[0 & 1] \
[1 & 0]
\end{pmatrix}, \quad
Y =
\begin{pmatrix}
[0 & -i] \
[i & 0]
\end{pmatrix}, \quad
Z =
\begin{pmatrix}
[1 & 0] \
[0 & -1]
\end{pmatrix}.
$$

- X gate: bit-flip ($|0\rangle \leftrightarrow |1\rangle$)
- Y gate: bit- and phase-flip
- Z gate: phase-flip ($|1\rangle \to -|1\rangle$)


Hadamard Gate

$$
H = \frac{1}{\sqrt{2}}
\begin{pmatrix}
[1 & 1] \
[1 & -1]
\end{pmatrix}.
$$

It creates superpositions:
$$
H|0\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}},
\qquad
H|1\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}.
$$

Rotation Gates

Rotations around the Bloch-sphere axes are parameterized by an angle $\phi$:

$$
R_x(\phi) =
\begin{pmatrix}
[\cos(\tfrac{\phi}{2}) & -i\sin(\tfrac{\phi}{2})] \
[-i\sin(\tfrac{\phi}{2}) & \cos(\tfrac{\phi}{2})]
\end{pmatrix},
$$

$$
R_y(\phi) =
\begin{pmatrix}
[\cos(\tfrac{\phi}{2}) & -\sin(\tfrac{\phi}{2})] \
[\sin(\tfrac{\phi}{2}) & \cos(\tfrac{\phi}{2})]
\end{pmatrix},
$$

$$
R_z(\phi) =
\begin{pmatrix}
[e^{-i\phi/2} & 0] \
[0 & e^{i\phi/2}]
\end{pmatrix}.
$$

Phase Gate

$$
P(\phi) =
\begin{pmatrix}
[1 & 0] \
[0 & e^{i\phi}]
\end{pmatrix}.
$$

It applies a phase to $|1\rangle$:
$$
P(\phi)|0\rangle = |0\rangle, \quad P(\phi)|1\rangle = e^{i\phi}|1\rangle.
$$

In [ ]:
# Identity and Pauli gates
def I2(): return np.eye(2, dtype=complex)
def X(): return np.array([[0, 1],[1, 0]], dtype=complex)
def Y(): return np.array([[0, -1j],[1j, 0]],dtype=complex)
def Z(): return np.array([[1, 0],[0, -1]], dtype=complex)

# Hadamard gate
def H(): return (1/np.sqrt(2)) * np.array([[1, 1],[1, -1]], dtype=complex)

# Rotation gates
def Rz(phi: float):
    return np.array([[np.exp(-1j*phi/2), 0],[0, np.exp(1j*phi/2)]], dtype=complex)

def Rx(phi: float):
    return np.array([[np.cos(phi/2), -1j*np.sin(phi/2)],[-1j*np.sin(phi/2), np.cos(phi/2)]], dtype=complex)

def Ry(phi: float):
    return np.array([[np.cos(phi/2), -np.sin(phi/2)],[np.sin(phi/2), np.cos(phi/2)]], dtype=complex)

def P(phi: float):
    "Phase gate"
    return np.array([[1, 0],[0, np.exp(1j*phi)]], dtype=complex)

## Section 3: Two-Qubit Gates

Two-qubit gates act on $\mathbb{C}^2 \otimes \mathbb{C}^2$ and are represented by $4\times4$ unitaries.

Controlled-NOT (CNOT)

$$
\mathrm{CNOT} =
\begin{pmatrix}
[1 & 0 & 0 & 0] \
[0 & 1 & 0 & 0] \
[0 & 0 & 0 & 1] \
[0 & 0 & 1 & 0]
\end{pmatrix}.
$$

Action:
$$
|c,t\rangle ;\mapsto; |c,,t \oplus c\rangle.
$$

Controlled-Z (CZ)

$$
\mathrm{CZ} =
\begin{pmatrix}
[1 & 0 & 0 & 0] \
[0 & 1 & 0 & 0] \
[0 & 0 & 1 & 0] \
[0 & 0 & 0 & -1]
\end{pmatrix}.
$$

Applies a phase $-1$ only to $|11\rangle$.

Controlled-U (CU)

For any $2\times2$ unitary $U$, the controlled version is

$$
\mathrm{CU} =
\begin{pmatrix}
[1 & 0 & 0 & 0] \
[0 & 1 & 0 & 0] \
[0 & 0 & U_{00} & U_{01}] \
[0 & 0 & U_{10} & U_{11}]
\end{pmatrix}.
$$

Applies $U$ to the target qubit only when the control is $|1\rangle$.

SWAP Gate

$$
\mathrm{SWAP} =
\begin{pmatrix}
[1 & 0 & 0 & 0] \
[0 & 0 & 1 & 0] \
[0 & 1 & 0 & 0] \
[0 & 0 & 0 & 1]
\end{pmatrix}.
$$

This exchanges the qubits:
$$
|a,b\rangle \to |b,a\rangle.
$$

iSWAP Gate

$$
\mathrm{iSWAP} =
\begin{pmatrix}
[1 & 0 & 0 & 0] \
[0 & 0 & i & 0] \
[0 & i & 0 & 0] \
[0 & 0 & 0 & 1]
\end{pmatrix}.
$$

Swaps the qubits and adds a phase $i$ to the exchanged states:
$$
|01\rangle \mapsto i|10\rangle, \quad |10\rangle \mapsto i|01\rangle.
$$

In [ ]:
def CNOT():
    return np.array([[1,0,0,0],
                     [0,1,0,0],
                     [0,0,0,1],
                     [0,0,1,0]], dtype=complex)
def CZ():
    return np.array([[1,0,0,0],
                     [0,1,0,0],
                     [0,0,1,0],
                     [0,0,0,-1]], dtype=complex)

def CU(U: np.ndarray):
    if U.shape != (2,2):
        raise ValueError("U must be a 2x2 unitary matrix.")
    return np.array([[1,0,0,0],
                     [0,1,0,0],
                     [0,0,U[0,0],U[0,1]],
                     [0,0,U[1,0],U[1,1]]], dtype=complex)

def SWAP():
    return np.array([[1,0,0,0],
                     [0,0,1,0],
                     [0,1,0,0],
                     [0,0,0,1]], dtype=complex)

def iSWAP():
    return np.array([[1,0,0,0],
                     [0,0,1j,0],
                     [0,1j,0,0],
                     [0,0,0,1]], dtype=complex)

## Measurement and visualization functions

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Random number generator
rng = np.random.default_rng(171)

# Compute basis probabilities from state vector
def basis_probabilities(psi: np.ndarray) -> np.ndarray:
    """
    Given a state vector |psi>, return the probabilities of measuring each basis state.
    The output is a 1D array of probabilities summing to 1.
    """
    probs = np.abs(psi)**2
    s = probs.sum()
    if not np.isclose(s, 1.0):
        probs = probs / s
    return probs

# Simulate measurement shots
def measure_shots(psi: np.ndarray, shots: int = 1000) -> dict:
    """
    Simulate measuring the state |psi> multiple times in the computational basis.
    Args:
        psi: 1D numpy array representing the state vector
        shots: number of measurement shots to simulate
    Returns:
        A dictionary mapping basis states (as bitstrings) to their counts.
    """

    probs = basis_probabilities(psi)
    dim = probs.size
    # Using numpy's choice to sample outcomes
    # This will give us a random outcome based on the probabilities
    outcomes = rng.choice(dim, size=shots, p=probs)
    # Count occurrences of each outcome
    n = int(np.log2(dim))
    # Format string for binary representation
    fmt = "{:0" + str(n) + "b}"
    counts = {}
    for o in outcomes:  
        b = fmt.format(o)
        counts[b] = counts.get(b, 0) + 1
    return counts

# ---------- Bloch sphere ----------
def state_to_angles(psi: np.ndarray):
    """Return (theta, phi) for |psi> = α|0> + β|1>."""
    psi = np.asarray(psi, dtype=complex).reshape(-1)
    if psi.size != 2:
        raise ValueError("psi must be a length-2 state vector.")
    psi = psi / np.linalg.norm(psi)
    alpha, beta = psi
    theta = 2 * np.arccos(np.clip(np.abs(alpha), 0, 1))
    phi = np.angle(beta) - np.angle(alpha)
    phi = (phi + np.pi) % (2*np.pi) - np.pi
    return theta, phi

def state_to_bloch(psi: np.ndarray):
    """Return (x, y, z, theta, phi) for |psi>."""
    theta, phi = state_to_angles(psi)
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    return x, y, z, theta, phi

def plot_bloch_point(psi: np.ndarray, title="Bloch Sphere: state"):
    x, y, z, theta, phi = state_to_bloch(psi)

    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111, projection='3d')

    # Bloch sphere wireframe
    u = np.linspace(0, 2*np.pi, 60)
    v = np.linspace(0, np.pi, 30)
    xs = np.outer(np.cos(u), np.sin(v))
    ys = np.outer(np.sin(u), np.sin(v))
    zs = np.outer(np.ones_like(u), np.cos(v))
    ax.plot_wireframe(xs, ys, zs, linewidth=0.5, alpha=0.3)

    # Coordinate axes
    ax.quiver(0,0,0, 1,0,0, color='r', linewidth=1, length=1)
    ax.quiver(0,0,0, 0,1,0, color='g', linewidth=1, length=1)
    ax.quiver(0,0,0, 0,0,1, color='b', linewidth=1, length=1)

    # Bloch vector
    ax.quiver(0,0,0, x,y,z, color='k', linewidth=3, length=1)

    # θ arc (polar angle)
    theta_arc = np.linspace(0, theta, 100)
    x_theta = np.sin(theta_arc) * np.cos(phi)
    y_theta = np.sin(theta_arc) * np.sin(phi)
    z_theta = np.cos(theta_arc)
    ax.plot(x_theta, y_theta, z_theta, 'm--', lw=1.5)
    ax.text(1.1*x_theta[len(x_theta)//2], 1.1*y_theta[len(y_theta)//2],
            1.1*z_theta[len(z_theta)//2],
            r'$\theta$ = ' + f'{np.degrees(theta):.1f}°', color='m', fontsize=13)

    # φ arc (azimuthal angle)
    phi_arc = np.linspace(0, phi, 100)
    x_phi = np.cos(phi_arc)
    y_phi = np.sin(phi_arc)
    z_phi = np.zeros_like(phi_arc)
    ax.plot(x_phi, y_phi, z_phi, 'c--', lw=1.5)
    ax.text(1.1*np.cos(phi/2), 1.1*np.sin(phi/2), 0,
            r'$\phi$ = ' + f'{np.degrees(phi):.1f}°', color='c', fontsize=13)

    # Labels and formatting
    ax.set_xlim([-1,1]); ax.set_ylim([-1,1]); ax.set_zlim([-1,1])
    ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
    ax.set_title(title + f"\nθ = {np.degrees(theta):.1f}°,  φ = {np.degrees(phi):.1f}°")
    ax.view_init(elev=25, azim=45)
    plt.tight_layout()
    return fig, ax
    

# ---------- Probability bar plot ----------
def plot_probabilities_bar(counts: dict, title="Measurement Probabilities"):
    labels = sorted(counts.keys())
    values = [counts[label] for label in labels]
    total = sum(values)
    probs = [v / total for v in values]

    fig, ax = plt.subplots(figsize=(8,5))
    ax.bar(labels, probs, color='skyblue', edgecolor='k')
    ax.set_ylim(0, 1)
    ax.set_ylabel('Probability')
    ax.set_title(title)
    for i, v in enumerate(probs):
        ax.text(i, v + 0.02, f"{v:.2f}", ha='center', fontsize=10)
    return fig, ax

## 1. First experiment:

We have said that appying unitary operators are equivalent to rotations on the Bloch sphere.

Try it out by representing $|0\rangle$ and appying the gates $X,Y,Z,H$

Also, create a custom superpositon state by defining 
$\alpha = 0.5, \beta = \sqrt(3)/2i$ and plot it. 

Then apply the same gates to $|1\rangle$

In [ ]:
ZERO = ket0(); ONE = ket1()
# Plot state 0
plot_bloch_point(ZERO, "Bloch Sphere: |0>")
plt.show()

In [ ]:
# Observe the action of the Paulis on |0>
psi_X = X() @ ZERO
plot_bloch_point(psi_X, "Bloch Sphere: X|0> = |1>")
plt.show()
psi_Y = Y() @ ZERO
plot_bloch_point(psi_Y, "Bloch Sphere: Y|0> = i|1>")
plt.show()
psi_Z = Z() @ ZERO
plot_bloch_point(psi_Z, "Bloch Sphere: Z|0> = |0>")
plt.show()
# Create |+> state
psi_plus = H() @ ZERO
plot_bloch_point(psi_plus, "Bloch Sphere: |+> = H|0>")
plt.show()
# Create custom superposition state
alpha = 1/2
beta = np.sqrt(3)/2 * 1j
psi_custom = superposition(alpha, beta)
plot_bloch_point(psi_custom, "Bloch Sphere: custom state")
plt.show()

In [ ]:
# Observe the action of the Paulis on |1>
psi_X1 = X() @ ONE
plot_bloch_point(psi_X1, "Bloch Sphere: X|1> = |0>")
plt.show()
psi_Y1 = Y() @ ONE
plot_bloch_point(psi_Y1, "Bloch Sphere: Y|1> = -i|0>")
plt.show()
psi_Z1 = Z() @ ONE
plot_bloch_point(psi_Z1, "Bloch Sphere: Z|1> = -|1>")
plt.show()

# Hadamard on |1>
psi_minus = H() @ ONE
plot_bloch_point(psi_minus, "Bloch Sphere: |-> = H|1>")
plt.show()  


## 2. Measure states

To verify that the quantum computer one of the eigenvalues with a probability of $|\alpha|$ or $|\beta|$. Let us measure some states:

$|+\rangle$ state = $\frac{1}{\sqrt{2}} (|0\rangle + |1\rangle)$ has theoretically 0.5 probability of every state.
Try measuring the $|+\rangle$ state for different number of shots = 10, 100, 1000, 10000

And see how the number of shots affects the accuracy.

In [ ]:
# Measure |+> state 1000 times
total_measures = []
shots = [10, 100, 500, 10000]
for s in shots:
    measures = []
    for i in range(30):  # Repeat 30 times for each shot count
        counts_plus = measure_shots(psi_plus, shots=s)
        measures.append(counts_plus)
    total_measures.append(measures)
# average results for each shot count
avg_measures = []
standard_deviations = []

for measures in total_measures:
    combined_counts = {}
    for counts in measures:
        for key, value in counts.items():
            combined_counts[key] = combined_counts.get(key, 0) + value
    avg_counts = {k: v / len(measures) for k, v in combined_counts.items()}
    avg_measures.append(avg_counts)
    
    # Calculate standard deviation
    shot_totals = [sum(counts.values()) for counts in measures]
    std_dev = {}
    for key in combined_counts.keys():
        values = [counts.get(key, 0) for counts in measures]
        mean = avg_counts[key]
        variance = sum((x - mean) ** 2 for x in values) / len(values)
        std_dev[key] = np.sqrt(variance)
    standard_deviations.append(std_dev)

# Plot average probabilities with error bars
for i, s in enumerate(shots):
    counts = avg_measures[i]
    std_dev = standard_deviations[i]
    labels = sorted(counts.keys())
    values = [counts[label] for label in labels]
    total = sum(values)
    probs = [v / total for v in values]
    errors = [std_dev[label] / total for label in labels]

    fig, ax = plt.subplots(figsize=(8,5))
    ax.bar(labels, probs, yerr=errors, capsize=5, color='lightgreen', edgecolor='k')
    ax.set_ylim(0, 1)
    ax.set_ylabel('Probability')
    ax.set_title(f'Measurement Probabilities for |+> state over {s} shots (averaged) and the error bars represent standard deviation')
    for j, v in enumerate(probs):
        ax.text(j, v + 0.02, f"{v:.2f}", ha='center', fontsize=10)
    plt.show()


## 3: Multi-Qubit States and the Bell State

In this experiment, we demonstrate how to build entangled states using tensor products and quantum gates.

We will create the Bell state (also known as the EPR pair):

$$
|\Phi^+\rangle = \frac{1}{\sqrt{2}} \left( |00\rangle + |11\rangle \right).
$$

This is one of the four maximally entangled Bell states and is produced by the following quantum circuit:

$$
|\Phi^+\rangle = \mathrm{CNOT}, (H \otimes I), |00\rangle.
$$

Steps:
1.	Start with both qubits in the ground state:
$$
|00\rangle = |0\rangle \otimes |0\rangle.
$$
2.	Apply a Hadamard gate on the first qubit to create a superposition:
$$
(H \otimes I)|00\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |10\rangle).
$$
3.	Apply a CNOT gate with the first qubit as control and the second as target:
$$
\mathrm{CNOT},(H \otimes I)|00\rangle
= \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle).
$$

The resulting state is the Bell state $|\Phi^+\rangle$.

Once built the circuit, measure the Bell state.

In [ ]:
# 3) Multi-qubit with tensor products + Bell state
I2_gate = I2()
CNOT_gate = CNOT()
H_gate = H()
# Create Bell state |Φ+> = (|00> + |11>)/√2
# |Φ+> = CNOT (H ⊗ I) |00>
circuit = CNOT_gate @ np.kron(H_gate, I2_gate)
psi_bell = circuit @ np.kron(ZERO, ZERO)
bell_counts = measure_shots(psi_bell, shots=10000)
print("Sampled counts for Bell state:", bell_counts)
plot_probabilities_bar(bell_counts, "Bell state |Φ+> probabilities")
plt.show()

## 4: Quantum Interference from a Phase Shift

This experiment demonstrates quantum interference by showing how a relative phase between basis states affects measurement outcomes in the X basis.

We start with a single qubit initialized in the state $|0\rangle$.
The circuit applies the following sequence:
1.	Hadamard gate — creates an equal superposition:
$$
H|0\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}}.
$$
2.	Phase gate $P(\phi)$ — adds a relative phase between $|0\rangle$ and $|1\rangle$:
$$
P(\phi) =
\begin{pmatrix}
[1 & 0] \
[0 & e^{i\phi}]
\end{pmatrix},
\qquad
P(\phi)H|0\rangle =
\frac{|0\rangle + e^{i\phi}|1\rangle}{\sqrt{2}}.
$$
3.	Hadamard gate (again) — brings the system back to the Z-basis (i.e., performs an X-basis measurement):
$$
H P(\phi) H |0\rangle.
$$

The probability of measuring the outcome $|0\rangle$ in this basis is:

$$
P(0) = \cos^2\left(\frac{\phi}{2}\right),
\qquad
P(1) = \sin^2\left(\frac{\phi}{2}\right).
$$

This sinusoidal dependence shows interference — constructive for $\phi = 0$, destructive for $\phi = \pi$.

In [ ]:
# --- helpers already in your notebook: ket0, H, P, basis_probabilities, measure_shots ---

def prob_zero_in_X_after_phase_measured(phi_vals, shots=4000):
    """Empirical P(0) from shot sampling after H -> P(phi) -> H, measuring in Z."""
    p0s = []
    for phi in phi_vals:
        psi = H() @ ket0()
        psi = P(phi) @ psi
        psi = H() @ psi
        counts = measure_shots(psi, shots=shots)  # Z-basis
        p0s.append(counts.get('0', 0) / shots)
    return np.array(p0s)

def prob_zero_in_X_after_phase_analytic(phi_vals):
    """Analytic P(0) via amplitudes after H -> P(phi) -> H (no sampling)."""
    p0s = []
    for phi in phi_vals:
        psi = H() @ ket0()
        psi = P(phi) @ psi
        psi = H() @ psi
        probs = basis_probabilities(psi)
        p0s.append(probs[0])
    return np.array(p0s)

def prob_zero_in_X_after_phase_theory(phi_vals):
    """Closed-form P(0) = cos^2(phi/2) = (1+cos phi)/2."""
    return np.cos(phi_vals/2.0)**2
    # equivalently: return 0.5 * (1 + np.cos(phi_vals))

# --- compute curves ---
phis = np.linspace(0, 2*np.pi, 200)
p_meas = prob_zero_in_X_after_phase_measured(phis, shots=10000)
p_an   = prob_zero_in_X_after_phase_analytic(phis)
p_th   = prob_zero_in_X_after_phase_theory(phis)

# (optional) sanity check: analytic == theory up to numerical precision
assert np.allclose(p_an, p_th, atol=1e-12), "Analytic and theoretical curves should match."

# --- plot together ---
plt.figure(figsize=(7,4.5))
plt.plot(phis, p_th, label="theory: cos²(φ/2)", linewidth=2)
plt.plot(phis, p_an, '--', label="analytic (amplitudes)")
plt.plot(phis, p_meas, '.', markersize=4, label="measured (shots)")
plt.xlabel("Phase φ (radians)")
plt.ylabel("P(measure 0 in X basis)")
plt.title("Interference: measured vs analytic vs theory")
plt.ylim(0, 1)
plt.legend()
plt.tight_layout()
plt.show()

## 5: One-Bit Quantum Phase Estimation (QPE)

This experiment estimates a single binary digit of the phase $\phi \in [0,1)$ for the unitary
$$
U = P(2\pi \phi) = 
\begin{pmatrix} 1 & 0 \\
0 & e^{i\,2\pi\phi} 
\end{pmatrix},
$$
acting on the target state $|1\rangle$, which is an eigenstate of U with eigenvalue $e^{i2\pi\phi}$.

Registers (left to right):
- Qubit 0 = ancilla ($|0\rangle$ initially)
- Qubit 1 = target  ($|1\rangle$ initially)

#### Circuit
1.	Prepare $|0\rangle_{\text{anc}} \otimes |1\rangle_{\text{tgt}}$.
2.	Hadamard on ancilla: $|0\rangle \mapsto \tfrac{|0\rangle + |1\rangle}{\sqrt{2}}$.
3.	Controlled-U with control=ancilla, target=target:
when ancilla is $|1\rangle$, apply $U = P(2\pi\phi)$ to target.
4.	Inverse QFT on 1 qubit = Hadamard on ancilla.
5.	Measure ancilla in the computational (Z) basis.

#### Probability and Estimator

Conditioned on the above, the ancilla’s outcome distribution is
$$
\Pr(\text{anc}=0) = \cos^2(\pi \phi), \qquad
\Pr(\text{anc}=1) = \sin^2(\pi \phi).
$$

With one ancilla, we learn one bit (the most significant bit at resolution 1/2):
- If $\phi \in [0, \tfrac12)$ → ancilla tends to 0 → estimate $\hat\phi \approx 0.0$.
- If $\phi \in [\tfrac12, 1)$ → ancilla tends to 1 → estimate $\hat\phi \approx 0.5$.

In [ ]:
def qpe_one_bit(phi: float, shots: int = 4000):
    """
    One-bit Quantum Phase Estimation for U = P(2πφ) on a single target qubit |1>.
    Uses one ancilla and returns:
      - counts_anc: {'0': n0, '1': n1} for the ancilla measurement
      - phase_bit: 0 or 1 (most likely bit)
      - phase_est: 0.0 if bit=0 else 0.5
      - raw_counts: full 2-qubit bitstrings counts
    Notes:
      P(θ) = diag(1, e^{iθ}); CU(P) = diag(1,1,1,e^{iθ}).
      Prob[ancilla=0] = cos^2(πφ), Prob[ancilla=1] = sin^2(πφ).
    """
    # Qubit order: ancilla = qubit 0 (MSB), target = qubit 1 (LSB)
    # Initial: |0>_a ⊗ |1>_t
    psi = kron_all(ket0(), ket1())

    # 1) H on ancilla
    psi = (kron_all(H(), I2()) @ psi)

    # 2) Controlled-U with U = P(2πφ) on target (control = ancilla)
    U = P(2*np.pi*phi)
    psi = (CU(U) @ psi)

    # 3) Inverse QFT on 1 qubit = H on ancilla
    psi = (kron_all(H(), I2()) @ psi)

    # 4) Measure both in Z; aggregate ancilla bit (left bit)
    raw_counts = measure_shots(psi, shots=shots)

    counts_anc = {'0': 0, '1': 0}
    for bits, c in raw_counts.items():
        counts_anc[bits[0]] = counts_anc.get(bits[0], 0) + c

    # Most likely bit and estimated phase (single-bit resolution)
    phase_bit = 0 if counts_anc['0'] >= counts_anc['1'] else 1
    phase_est = 0.0 if phase_bit == 0 else phase_bit * 0.5

    return counts_anc, phase_bit, phase_est, raw_counts

counts_anc, bit, est, raw = qpe_one_bit(phi=0.3, shots=8000)
print("Ancilla counts:", counts_anc)
print("Estimated bit:", bit, " -> phase ≈", est)

To visualize how the ancilla probability follows $\cos^2(\pi \phi)$, sweep $\phi$ and plot measured vs. theoretical curves.

In [ ]:
def qpe_one_bit_sweep(phis, shots=4000):
    """
    For each phi in [0,1), run one-bit QPE and collect:
      - phase_estimates (0.0 or 0.5)
      - measured P(anc=0) and P(anc=1)
    Returns arrays: est, p0_meas, p1_meas
    """
    est = []
    p0_meas = []
    p1_meas = []
    for phi in phis:
        counts_anc, bit, phase_est, _ = qpe_one_bit(phi=phi, shots=shots)
        total = counts_anc['0'] + counts_anc['1']
        p0 = counts_anc['0'] / total
        p1 = counts_anc['1'] / total
        est.append(phase_est)
        p0_meas.append(p0)
        p1_meas.append(p1)
    return np.array(est), np.array(p0_meas), np.array(p1_meas)

# parameters
phis = np.linspace(0.0, 1.0, 61, endpoint=False)  # φ in [0,1)
shots = 4000

# run sweep
est, p0_meas, p1_meas = qpe_one_bit_sweep(phis, shots=shots)

# theory curves
p0_theo = np.cos(np.pi * phis)**2
p1_theo = 1.0 - p0_theo  # sin^2(πφ)

# --- plots ---
fig, axes = plt.subplots(2, 1, figsize=(8, 8), sharex=True)

# (A) Estimated phase vs true phase
ax = axes[0]
ax.plot(phis, phis, '--', label='y = φ (identity)', alpha=0.6)  # reference
ax.step(phis, est, where='mid', label='estimated phase (one-bit QPE)')
ax.hlines([0.0, 0.5], xmin=phis.min(), xmax=phis.max(), colors=['C1','C2'], linestyles=':', alpha=0.5)
ax.vlines([0.25, 0.75], ymin=-0.05, ymax=1, colors='k', linestyles=':', alpha=0.3, label='decision boundaries')
ax.set_ylabel('Estimated phase')
ax.set_title('One-Bit QPE: Estimated Phase vs True φ')
ax.set_ylim(-0.05, 1)
ax.legend(loc='upper left')

# (B) Ancilla probability curves (measured vs theory)
ax = axes[1]
ax.plot(phis, p0_theo, label='theory P(anc=0)=cos²(πφ)', linewidth=2)
ax.plot(phis, p1_theo, label='theory P(anc=1)=sin²(πφ)', linewidth=2)
ax.plot(phis, p0_meas, 'o', markersize=4, label='measured P(anc=0)')
ax.plot(phis, p1_meas, 'o', markersize=4, label='measured P(anc=1)')
ax.vlines([0.25, 0.75], ymin=0.0, ymax=1.0, colors='k', linestyles=':', alpha=0.3)
ax.set_xlabel('φ')
ax.set_ylabel('Probability')
ax.set_title('Ancilla Outcome Probabilities vs φ')
ax.set_ylim(0, 1)
ax.legend(ncols=2)
ax.grid(alpha=0.15)

plt.tight_layout()
plt.show()

If we take the intersection point between the real phase and the estimated phase

In [ ]:
counts_anc, bit, est, raw = qpe_one_bit(phi=0.5, shots=8000)
print("Ancilla counts:", counts_anc)
print("Estimated bit:", bit, " -> phase ≈", est)